In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import xml.etree.ElementTree as xet
from collections import defaultdict
from sklearn.cluster import KMeans
import pandas as pd
import albumentations as A
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf # 2.17.0
print(tf.__version__)
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D, Add, concatenate, Lambda
from tensorflow.keras.models import Model
import struct
import importlib.util
import sys
import os
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Set the random seed for reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# Define the path to your script
utils_path = '/kaggle/input/project-numberplate/utils.py'

# Load the module
spec = importlib.util.spec_from_file_location("utils", utils_path)
utils = importlib.util.module_from_spec(spec)
sys.modules["utils"] = utils
spec.loader.exec_module(utils)
from utils import WeightReader, freeze_yolov3_layers, process_outputs, draw_boxes, make_yolov3_model, CustomLoss, _conv_block, yolo_head, trueBoundingBoxTransformer, YOLOv3Metrics, crop_and_resize, bbox_to_center_and_size, img_bbox_resize, calculate_iou, multi_output_image_generator

In [ ]:
#Load the df if saved earlier
#df = pd.read_pickle('/kaggle/input/project-numberplate/data.pkl')
#df.head()

In [ ]:
# For reproducibility, mention folder paths in the below order
# Directory where your files are located
folder_path0 = '/kaggle/input/project-numberplate/image_data/image_data/State-wise_OLX'
folder_path1 = '/kaggle/input/project-numberplate/image_data/image_data/google_images'
folder_path2 = '/kaggle/input/project-numberplate/image_data/image_data/more_data'
folder_path3 = '/kaggle/input/project-numberplate/image_data/image_data/video_images'

# List of directories where your files are located
source_dirs = [folder_path0, folder_path1, folder_path2, folder_path3]

# Initialize lists for image and xml files
image_files = []
xml_files = []

# Gather files from all source directories
for folder_path in source_dirs:
    image_files.extend([os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in ['.png', '.jpeg', '.jpg']])
    xml_files.extend([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xml')])

# Create defaultdict to group files by base name
file_groups = defaultdict(lambda: {'image': None, 'xml': None})

# Fill defaultdict with image files
for image_file in image_files:
    base_name = os.path.splitext(os.path.basename(image_file))[0]  # Get base name without extension
    file_groups[base_name]['image'] = image_file

# Fill defaultdict with xml files
for xml_file in xml_files:
    base_name = os.path.splitext(os.path.basename(xml_file))[0]  # Get base name without extension
    file_groups[base_name]['xml'] = xml_file

# Filter out groups where both image and xml files are present
file_pairs = [(group['image'], group['xml']) for group in file_groups.values() if group['image'] and group['xml']]

# Displaying the result
print(len(file_pairs))

data_dict = {'img':[], 'targ':[]}
for img, filename in file_pairs:
    tree = xet.parse(filename)
    root = tree.getroot()
    bboxes = []
    for object_ in root.findall('object'):
        labels = object_.find('bndbox')
        if labels is not None:
            xmin = int(labels.find('xmin').text)
            ymin = int(labels.find('ymin').text)
            xmax = int(labels.find('xmax').text)
            ymax = int(labels.find('ymax').text)
            bboxes.append(bbox_to_center_and_size(xmin, ymin, xmax, ymax))
    img_, out_bbox = img_bbox_resize(img, bboxes, targetsize = 608)
    data_dict['img'].append(img_)
    data_dict['targ'].append(out_bbox)

df = pd.DataFrame(data_dict)
rows_to_drop = []
for m in range(df.shape[0]):
    bboxes = df.iloc[m,1]
    for bbox in bboxes:
        x_center, y_center, w, h = bbox
        x_min, y_min, x_max, y_max = x_center - w/2, y_center - h/2, x_center + w/2, y_center + h/2
        if x_min < 0 or y_min < 0 or x_max > 1 or y_max > 1:
            rows_to_drop.append(m)
df.drop(rows_to_drop, inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape
#Save the dataframe for direct loading of preprocessed data
#df.to_pickle('data.pkl') 

In [ ]:
X = df[['img']]
y = df['targ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df.shape

In [ ]:
test_df.shape

In [ ]:
# Define the transformation pipeline with bbox_params
transform = A.Compose([
    A.OneOf([
        A.Resize(height=203, width=203, p= 1.0),  # Resize to a smaller size to simulate distance
        A.NoOp(p=1.0)
    ], p=1.0),

    A.PadIfNeeded(min_height=608, min_width=608, border_mode=cv2.BORDER_CONSTANT, value=[0, 0, 0]),  # Pad back to original size
    A.Rotate(limit=10, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.GaussNoise(p=0.2),
    A.MotionBlur(p=0.2),
    A.Perspective(scale=(0.05, 0.1), p=0.5),
], bbox_params=A.BboxParams(format='yolo', min_area=0.1, min_visibility=0.3, label_fields=['labels']))


# Generate multiple augmented images
ini_size = train_df.shape[0]
for m in range(ini_size):
    image = train_df.iloc[m,0]
    bboxes = train_df.iloc[m,1]
    labels = [0]*len(bboxes) #dummy labels
    num_augmentations = 1
    for i in range(num_augmentations):
        transformed = transform(image=image, bboxes=bboxes, labels=labels)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        new_row = {'img': transformed_image, 'targ': transformed_bboxes}
        train_df.loc[len(train_df)] = new_row

In [ ]:
ini_size = train_df.shape[0]
for m in range(ini_size):
    image = train_df.iloc[m,0]
    bboxes = train_df.iloc[m,1]
    num_augmentations = 1
    for i in range(num_augmentations):
        if len(bboxes) > 0:
            resized_image, transformed_bbox, valid = crop_and_resize(image, bboxes[0], output_size=(608, 608), zoom_range=(1.4, 2.8), thresh_area= (0.01, 0.2))
            if valid:
                new_row = {'img': resized_image, 'targ': [transformed_bbox]}
                train_df.loc[len(train_df)] = new_row

In [ ]:
train_df.shape

In [ ]:
normalized_bboxes_list = []
for a in train_df.iloc[:,1]:
    for k in a:
        normalized_bboxes_list.append(k)
normalized_bboxes = np.array(normalized_bboxes_list)
normalized_wh = normalized_bboxes[:,2:]
cluster_anchors = {}
average_iou = []
for num_anchors in range(1,16):
    kmeans = KMeans(n_clusters=num_anchors, random_state=42, verbose = 0).fit(normalized_wh)
    anchor_boxes = kmeans.cluster_centers_
    cluster_anchors[num_anchors] = anchor_boxes
# Evaluate the anchor boxes
    ious = []
    for bbox in normalized_wh:
        best_iou = 0
        for anchor in anchor_boxes:
            iou = calculate_iou(bbox, anchor)
            if iou > best_iou:
                best_iou = iou
        ious.append(best_iou)
    average_iou.append(np.mean(ious))
    
plt.figure(figsize=(10, 6))
plt.plot(range(1,16),average_iou, marker='o', color='blue')
plt.title('Average IoU vs. Number of Clusters (Anchor Boxes)')
plt.ylabel('Average IoU')
plt.xlabel('Clusters')
plt.xticks(range(1, 16))
plt.grid(True)
plt.show()

In [ ]:
anchors = cluster_anchors[4]
#np.save('anchors.npy', anchors)
# Load the NumPy array from the file
#anchors = np.load('/kaggle/input/project-numberplate/anchors.npy')
#print("Loaded anchors:")
print(anchors)

In [ ]:
batch_size = 32
# Calculate the number of samples needed
num_samples = len(train_df)
remainder = num_samples % batch_size
if remainder != 0:
    samples_needed = batch_size - remainder

    # Randomly sample from the DataFrame
    random_samples = train_df.sample(n=samples_needed, replace=True, random_state=42)

    # Append the samples to the original DataFrame
    train_df = pd.concat([train_df, random_samples]).reset_index(drop=True)

print(f"New number of samples: {len(train_df)}")  # Should be a multiple of batch_size

In [ ]:
model = make_yolov3_model()
#model.summary()

In [ ]:
weight_reader = WeightReader('/kaggle/input/project-numberplate/yolov3.weights')
weight_reader.load_weights(model)

In [ ]:
# Input tensor
inputs = model.input

x = model.get_layer('leaky_80').output
yolo_82 = _conv_block(x, [{'filter':  20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

x = model.get_layer('leaky_92').output
yolo_94 = _conv_block(x, [{'filter': 20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

x = model.get_layer('leaky_104').output
yolo_106 = _conv_block(x, [{'filter': 20, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)


# Create a new model with the modified architecture
model = Model(inputs=inputs, outputs=[yolo_82, yolo_94, yolo_106])

In [ ]:
model.summary()

In [ ]:
# Freeze all layers except those connected to outputs
for layer in model.layers:
    layer.trainable = False
# Ensure the output layers are trainable
model.get_layer('conv_81').trainable = True
model.get_layer('conv_93').trainable = True
model.get_layer('conv_105').trainable = True

custom_loss13 = CustomLoss(anchors = anchors, box_weight = 10.0)
custom_loss26 = CustomLoss(anchors = anchors, box_weight = 10.0)
custom_loss52 = CustomLoss(anchors = anchors, box_weight = 10.0)

optimizer = tf.keras.optimizers.Adam(learning_rate =0.001)
model.compile(optimizer = optimizer, loss = [custom_loss13, custom_loss26, custom_loss52])

In [ ]:
transformer = trueBoundingBoxTransformer(anchors, target_sizes = [19,38,76])
y_13, y_26, y_52 = transformer.transform(train_df)
y_13.shape

In [ ]:
batch_size = 32
steps_per_epoch = len(train_df) // batch_size
train_gen = multi_output_image_generator(train_df, y_13, y_26, y_52, batch_size)

In [ ]:
history = model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=10, initial_epoch = 0)

In [ ]:
for i, layer in enumerate(model.layers):
    if i > 54:  # Assuming the index starts from 0
        layer.trainable = True
optimizer = tf.keras.optimizers.Adam(learning_rate =0.0001)
model.compile(optimizer = optimizer, loss = [custom_loss13, custom_loss26, custom_loss52])

In [ ]:
history = model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=40, initial_epoch = 10)

In [ ]:
test_images = np.stack(test_df['img'].values)
ytest_13, ytest_26, ytest_52 = transformer.transform(test_df)
Metrics = YOLOv3Metrics([anchors,anchors,anchors], confidence_thresh=0.5, iou_thresh = 0.7, NMS_thresh=0.0)
test_images.shape

In [ ]:
metrics_ = {'TP':0,'FP':0,'FN':0}
for m in range(test_images.shape[0]):
    pred = model.predict(test_images[m:m+1], verbose=0)
    tp, fp, fn = Metrics.metrics(pred, ytest_13[m:m+1])
    metrics_['TP'] += tp
    metrics_['FP'] += fp
    metrics_['FN'] += fn
metrics_

In [ ]:
model.save('model_40.keras')